### 기본 라이브러리 불러오기

In [1]:
# 라이브러리 임포트
import os
import pandas as pd
import chardet

# 사용자 정의함수 경로 설정
import sys
sys.path.append("G:/내 드라이브/Source/_Custom_Function")
# 사용자 정의함수 불러오기

import time

import ExcelFile_Merge as em
import ExcelFile_Read as er
import ExcelFile_EDA as ee

In [ ]:
df_sum = pd.DataFrame()

In [ ]:
# _years = ['2020', '2021', '2022', '2023', '2024']
_years = ['2021']

In [2]:
path_dateset = 'G:\\내 드라이브\\DataSet\\'
path_sub = '서울시 차량통행속도(월별)\\'

---
### 서울시 차량통행속도

- 2020년 1월 서울시 차량통행속도 ~ 2020년 12월 서울시 차량통행속도
- 2021년 1월 서울시 차량통행속도 ~ 2021년 12월 서울시 차량통행속도
- 2022년 1월 서울시 차량통행속도 ~ 2022년 12월 서울시 차량통행속도
- 2023년 1월 서울시 차량통행속도 ~ 2023년 12월 서울시 차량통행속도
- 2024년 1월 서울시 차량통행속도 ~ 2024년 12월 서울시 차량통행속도

In [ ]:
for cnt, _year in enumerate(_years):
    
    path = path_dateset + path_sub + _year + '\\'
    savefile = '서울시_차량통행속도_' + _year   # Ex) 서울시_차량통행속도_2024
    print(f'path : {path}, savefile : {savefile}')
    
    # 파일 경로명 변경
    file_list = os.listdir(path)
    file_lists = [file for file in file_list if file.endswith((".csv", ".xlsx"))]
    file_lists.sort()
    print('file_lists : \n', '\n '.join(file_lists))
    print('===========================================')

    # try:
    for _, filename in enumerate(file_lists):        
        print('filename : ', filename + ', encoding : ', em.Get_ExcelEncoding(path + filename))
        start_time_full = time.time()

        # 서울 생활이동 데이터 불러오기
        start_time = time.time()
        print('path + filename : ', path + filename)
        
        dask =  pd.read_excel(path + filename)
        print("Read_dask(" + filename + ") time :", time.time() - start_time)

        df_dask_rtn = dask
        del dask  # 메모리 정리
        
        df_sum = pd.concat([df_dask_rtn, df_sum], axis = 0)
        del df_dask_rtn  # 메모리 정리

    # for문 종료 후, 실행
    df_sum.to_parquet(path + savefile + '.parquet', index=False)
    del df_sum
    
    # except:
    #     print('- Err : ', savefile)  
    

---
# 데이터 추가병합 (20~ 24년 출/퇴근 기준으로 병합)

In [3]:
start_time = time.time()
df_BikeRent_2020 = pd.read_parquet('G:\\내 드라이브\\DataSet\\서울시 차량통행속도(월별)\\병합_년별\\서울시_차량통행속도_2020.parquet')
df_BikeRent_2021 = pd.read_parquet('G:\\내 드라이브\\DataSet\\서울시 차량통행속도(월별)\\병합_년별\\서울시_차량통행속도_2021.parquet')
df_BikeRent_2022 = pd.read_parquet('G:\\내 드라이브\\DataSet\\서울시 차량통행속도(월별)\\병합_년별\\서울시_차량통행속도_2022.parquet')
df_BikeRent_2023 = pd.read_parquet('G:\\내 드라이브\\DataSet\\서울시 차량통행속도(월별)\\병합_년별\\서울시_차량통행속도_2023.parquet')
df_BikeRent_2024 = pd.read_parquet('G:\\내 드라이브\\DataSet\\서울시 차량통행속도(월별)\\병합_년별\\서울시_차량통행속도_2024.parquet')
print("read_csv time :", time.time() - start_time)

read_csv time : 8.27233624458313


In [4]:
df_BikeRent_2020.shape[0]

1673549

In [5]:
df_BikeRent_2020.head()

,일자,요일,도로명,링크아이디,시점명,종점명,방향,거리,차선수,기능유형구분,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,20200901,화,4.19로,1080012200,아카데미하우스,국립4.19묘지,상행,1179,1.0,보조간선도로,...,36.67,38.17,43.36,47.62,34.24,37.29,30.62,46.11,44.20,59.19
1,20200901,화,4.19로,1080012800,국립4.19묘지,국립4.19묘지입구,상행,352,2.0,보조간선도로,...,11.22,10.66,10.81,9.31,9.00,12.08,12.77,16.94,16.43,16.02
2,20200901,화,4.19로,1080012700,국립4.19묘지입구,국립4.19묘지,하행,354,2.0,보조간선도로,...,21.38,20.95,24.90,22.15,18.85,17.36,19.45,19.99,25.99,29.09
3,20200901,화,4.19로,1080012100,국립4.19묘지,아카데미하우스,하행,1190,1.0,보조간선도로,...,43.59,44.07,62.28,61.14,47.81,50.33,46.09,49.38,51.10,39.82
4,20200901,화,가락로,1230024700,방이초교,방이동고분군,상행,844,1.0,기타도로,...,13.02,15.21,14.10,14.69,14.11,14.40,15.19,15.06,15.15,17.67


In [6]:
df_BikeRent_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1673549 entries, 0 to 1673548
Data columns (total 36 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   일자       1673549 non-null  int64  
 1   요일       1673549 non-null  object 
 2   도로명      1673549 non-null  object 
 3   링크아이디    1673549 non-null  int64  
 4   시점명      1669805 non-null  object 
 5   종점명      1669805 non-null  object 
 6   방향       1673549 non-null  object 
 7   거리       1673549 non-null  int64  
 8   차선수      1672712 non-null  float64
 9   기능유형구분   1673549 non-null  object 
 10  도심/외곽구분  1673549 non-null  object 
 11  권역구분     1673549 non-null  object 
 12  01시      1663437 non-null  float64
 13  02시      1663422 non-null  float64
 14  03시      1663257 non-null  float64
 15  04시      1658299 non-null  float64
 16  05시      1653353 non-null  float64
 17  06시      1657794 non-null  float64
 18  07시      1662921 non-null  float64
 19  08시      1667811 non-null  float64
 20  09

In [7]:
df_BikeRent_2020['요일'].value_counts()

요일
수    240094
금    240078
일    240077
토    240066
목    240058
월    237841
화    235335
Name: count, dtype: int64

In [8]:
pd.set_option('display.max_columns', None)

In [ ]:
#df_BikeRent_2024.columns

# _columns = ['01시', '02시', '03시', '04시', '05시', '06시', '07시', '08시', '09시', '10시',
#        '11시', '12시', '13시', '14시', '15시', '16시', '17시', '18시', '19시', '20시',
#        '21시', '22시', '23시', '24시']
# df_BikeRent_2024 = df_BikeRent_2024.drop(_columns, axis=1)

# df_BikeRent_2024 = df_BikeRent_2024.rename(columns={'~01시': '01시', '~02시': '02시', '~03시': '03시', '~04시': '04시', '~05시': '05시', 
#                                                     '~06시': '06시', '~07시': '07시', '~08시': '08시', '~09시': '09시', '~10시': '10시', 
#                                                     '~11시': '11시', '~12시': '12시', '~13시': '13시', '~14시': '14시', '~15시': '15시', 
#                                                     '~16시': '16시', '~17시': '17시', '~18시': '18시', '~19시': '19시', '~20시': '20시', 
#                                                     '~21시': '21시', '~22시': '22시', '~23시': '23시', '~24시': '24시', })
#df_BikeRent_2024.to_parquet('G:\\내 드라이브\\DataSet\\서울시 차량통행속도(월별)\\병합_년별\\서울시_차량통행속도_2024.parquet', index=False)

In [9]:
df_BikeRent_2024.head(2)

,일자,요일,도로명,링크아이디,시점명,종점명,방향,거리,차선수,기능유형구분,도심/외곽구분,권역구분,01시,02시,03시,04시,05시,06시,07시,08시,09시,10시,11시,12시,13시,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,20240901,일,4.19로,1080012200,아카데미하우스,국립4.19묘지,상행,1179,1,보조간선도로,외곽,강북구,46.35,41.84,48.43,57.67,44.61,37.18,42.80,44.45,35.04,36.71,40.81,39.74,35.38,27.05,31.10,28.31,29.22,32.31,28.59,35.42,28.63,51.61,44.03,51.40
1,20240901,일,4.19로,1080012800,국립4.19묘지,국립4.19묘지입구,상행,352,1,보조간선도로,외곽,강북구,24.47,24.93,20.45,24.22,18.22,18.69,19.03,12.37,13.67,15.41,16.97,12.37,15.75,13.25,11.23,12.31,12.36,13.20,14.31,18.20,15.83,18.15,25.56,19.72


---
### 출근_시, 퇴근_시 컬럼 추가

In [10]:
df_sum = pd.DataFrame()

dfs = [df_BikeRent_2020, df_BikeRent_2021, df_BikeRent_2022, df_BikeRent_2023, df_BikeRent_2024]

for df in dfs:
    df_출근 = df_퇴근 = []

    #print(df.columns)
    # 20240101 => 2024
    df['일자'] = df['일자'].astype(str)
    df['대상연도'] = df['일자'].str.slice(start = 0, stop=4)

    df['출근_시'] = df['07시'] + df['08시']    # 출근(07 ~ 09)
    df['퇴근_시'] = df['17시'] + df['18시']    # 퇴근(17 ~ 19)

    # 평일(월 ~ 금) 데이터만 가져오기
    df = df[df['요일'].isin(['월', '화', '수', '목', '금'])]

    df_출근 = df_퇴근 = df.groupby(['권역구분', '대상연도']).agg({'출근_시': 'mean', '퇴근_시': 'mean'}).reset_index()

    df_출근['출퇴근'] = '출근'
    df_출근 = df_출근.drop(['퇴근_시'], axis=1)
    df_출근 = df_출근.rename(columns={'출근_시': '평균속도'})

    df_퇴근['출퇴근'] = '퇴근'
    df_퇴근 = df_퇴근.drop(['출근_시'], axis=1)
    df_퇴근 = df_퇴근.rename(columns={'퇴근_시': '평균속도'})

    df_출근 = df_출근[['대상연도', '출퇴근', '권역구분', '평균속도']]
    df_퇴근 = df_퇴근[['대상연도', '출퇴근', '권역구분', '평균속도']]
    
    df_sum = pd.concat([df_sum, df_출근, df_퇴근], axis = 0)
    del df_출근, df_퇴근  # 메모리 정리

In [22]:
cond1 = df_sum['대상연도'] == '2024'
cond2 = df_sum['출퇴근'] == '퇴근'
df_sum[cond1 & cond2].count()

대상연도    25
출퇴근     25
권역구분    25
평균속도    25
dtype: int64

In [ ]:
df_sum.to_parquet('G:\\내 드라이브\\DataSet\\_파킷 파일\\연도별  자치구별 서울시_차량통행속도_2020_2024.parquet', index=False)

In [ ]:
start_time = time.time()
df = pd.read_parquet('G:\\내 드라이브\\DataSet\\_파킷 파일\\연도별  자치구별 서울시_차량통행속도_2020_2024.parquet')
print("read_csv time :", time.time() - start_time)
df.head()

read_csv time : 0.11508965492248535


,대상연도,출퇴근,권역구분,평균속도
0,2020,출근,강남구,64.142934
1,2020,출근,강동구,53.637487
2,2020,출근,강북구,49.646329
3,2020,출근,강서구,56.206727
4,2020,출근,관악구,63.636772
